#### Import Necessary Libraries

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import os
import seaborn as sns

adbCodes=pd.read_csv(os.path.join(os.getcwd(),"data","ADB_CountryCodes.csv"))
NewData=adbCodes[['name','ISO','ADB_DEP','TE_Name','CurrencyCode']].copy()
NewData[['currency_Price','currency_YOYChange','currency_Date']]=""
NewData[['CPI_Inflation_YoY','Inflation_Change','CPI_Inflation_Date']]=""
NewData[['Food_Inflation_YoY','Food_Inflation_Change','Food_Inflation_Date']]=""



#### Download Currency data from Trading Economics

In [3]:
#Set Necessary headers to read data
headers = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.182 Safari/537.36"}

for i in range(0,NewData.shape[0]):
    URL="https://tradingeconomics.com/"+NewData['TE_Name'].iloc[i]+"/currency"
    response=requests.get(URL,headers=headers)
    soup=BeautifulSoup(response.content,'html.parser')
    tables=soup.find_all('table','table')
    if len(tables)>2:
        df1=pd.read_html(str(tables[1]))[0]
        if 'Crosses' in list(df1.columns):
            NewData['currency_Price'].iloc[i]=float(df1[df1['Crosses']=="USD"+NewData['CurrencyCode'].iloc[i]]['Price'])
            NewData['currency_YOYChange'].iloc[i]=df1[df1['Crosses']=="USD"+NewData['CurrencyCode'].iloc[i]]['Year'].iloc[0]
            NewData['currency_Date'].iloc[i]=df1[df1['Crosses']=="USD"+NewData['CurrencyCode'].iloc[i]]['Date'].iloc[0]
    print(i,end=" ")
            
            
            

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 

#### Download Inflation, CPI Change Data from Trading Economics

In [4]:
#Set Necessary headers to read data
headers = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.182 Safari/537.36"}
for i in range(0,NewData.shape[0]):
    URL="https://tradingeconomics.com/"+NewData['TE_Name'].iloc[i]+"/inflation-cpi"
    response=requests.get(URL,headers=headers)
    soup=BeautifulSoup(response.content,'html.parser')
    tables=soup.find_all('table','table')
    tableno=""
    current=""
    prev=""
    change=""
    if len(tables)>=1:
        for j in range(0,len(tables)):
            df1=pd.read_html(str(tables[j]))[0]
            if 'Related' in list(df1.columns):
                tableno=j
                df1=pd.read_html(str(tables[tableno]))[0]
                current=df1[df1['Related']=="Inflation Rate"]['Last'].iloc[0]
                prev=df1[df1['Related']=="Inflation Rate"]['Previous'].iloc[0]
                change=current-prev 
                NewData['CPI_Inflation_YoY'].iloc[i]=current
                NewData['Inflation_Change'].iloc[i]=change
                NewData['CPI_Inflation_Date'].iloc[i]=df1[df1['Related']=="Inflation Rate"]['Reference'].iloc[0]
                print(i, end=" ")
                
            

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 16 23 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 45 

#### Download Food Inflation Data from Trading Economics

In [5]:
#Set Necessary headers to read data
headers = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.182 Safari/537.36"}
for i in range(0,NewData.shape[0]):
    URL="https://tradingeconomics.com/"+NewData['TE_Name'].iloc[i]+"/food-inflation"
    response=requests.get(URL,headers=headers)
    soup=BeautifulSoup(response.content,'html.parser')
    tables=soup.find_all('table','table')
    tableno=""
    current=""
    prev=""
    change=""
    if len(tables)>=1:
        for j in range(0,len(tables)):
            df1=pd.read_html(str(tables[j]))[0]
            if 'Related' in list(df1.columns):
                tableno=j
                df1=pd.read_html(str(tables[tableno]))[0]
                current=df1[df1['Related']=="Food Inflation"]['Last'].iloc[0]
                prev=df1[df1['Related']=="Food Inflation"]['Previous'].iloc[0]
                change=current-prev 
                NewData['Food_Inflation_YoY'].iloc[i]=current
                NewData['Food_Inflation_Change'].iloc[i]=change
                NewData['Food_Inflation_Date'].iloc[i]=df1[df1['Related']=="Food Inflation"]['Reference'].iloc[0]
                print(i, end=" ")
                

0 1 2 3 4 5 6 7 9 10 11 12 13 14 16 23 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 45 

#### Download Commodity Prices

In [55]:
url="https://thedocs.worldbank.org/en/doc/5d903e848db1d1b83e0ec8f744e55570-0350012021/related/CMO-Historical-Data-Monthly.xlsx"
r = requests.get(url)
open('temp.xls', 'wb').write(r.content)
df = pd.read_excel('temp.xls',sheet_name="Monthly Prices",skiprows=4, usecols="A:BT")
df.set_index("Unnamed: 0",inplace=True)

In [56]:
df.tail(1)

,"Crude oil, average","Crude oil, Brent","Crude oil, Dubai","Crude oil, WTI","Coal, Australian","Coal, South African","Natural gas, US","Natural gas, Europe","Liquefied natural gas, Japan",Natural gas index,...,Aluminum,"Iron ore, cfr spot",Copper,Lead,Tin,Nickel,Zinc,Gold,Platinum,Silver
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
2022M05,110.096667,112.37,108.32,109.6,..,280,8.1373,29.84629,16.5267,251.899449,...,2830.32,131.21,9377.15,2142.48,35769.39,28062.55,3751.48,1848.5,958.38,21.905


In [134]:
ColsEnergy=['Crude oil, average','Natural gas, US']
UnitsEnergy=list(df[ColsEnergy].iloc[0])

ColsOils=['Palm oil', 'Soybean oil','Sunflower oil']
UnitsOils=list(df[ColsOils].iloc[0])

ColsCereals=['Soybeans','Maize','Rice, Thai 5% ', 'Rice, Thai 25% ', 'Rice, Thai A.1',
       'Rice, Viet Namese 5%', 'Wheat, US SRW', 'Wheat, US HRW']
UnitsCereals=list(df[ColsCereals].iloc[0])

ColsOthers=['Sugar, world','Cotton, A Index']
UnitsOthers=list(df[ColsOthers].iloc[0])

ColsFert=['Phosphate rock', 'DAP', 'TSP', 'Urea ', 'Potassium chloride']
UnitsFert=list(df[ColsFert].iloc[0])

CommodityTable=pd.DataFrame(columns = ['Group', 'Title','Units', 'LastDate','Price','MoM', 'YoY']+list(map(str, range(13))))

for i in ColsEnergy:
    cat="Energy"
    name=i
    units=df[i].iloc[0]
    LastDate=df[i].tail(1).index[0]
    price=float(df[i].tail(1))
    mom=(df[i].iloc[-1] -df[i].iloc[-2])/df[i].iloc[-2]*100
    yoy=(df[i].iloc[-1] -df[i].iloc[-13])/df[i].iloc[-13]*100
    a_series = pd.Series([cat,name,units,LastDate,price,mom,yoy]+list(df[i].iloc[-13:]), index=CommodityTable.columns)
    CommodityTable = CommodityTable.append(a_series, ignore_index=True)

for i in ColsOils:
    cat="Oil"
    name=i
    units=df[i].iloc[0]
    LastDate=df[i].tail(1).index[0]
    price=float(df[i].tail(1))
    mom=(df[i].iloc[-1] -df[i].iloc[-2])/df[i].iloc[-2]*100
    yoy=(df[i].iloc[-1] -df[i].iloc[-13])/df[i].iloc[-13]*100
    a_series = pd.Series([cat,name,units,LastDate,price,mom,yoy]+list(df[i].iloc[-13:]), index=CommodityTable.columns)
    CommodityTable = CommodityTable.append(a_series, ignore_index=True)

for i in ColsCereals:
    cat="Cereals and Grains"
    name=i
    units=df[i].iloc[0]
    LastDate=df[i].tail(1).index[0]
    price=float(df[i].tail(1))
    mom=(df[i].iloc[-1] -df[i].iloc[-2])/df[i].iloc[-2]*100
    yoy=(df[i].iloc[-1] -df[i].iloc[-13])/df[i].iloc[-13]*100
    a_series = pd.Series([cat,name,units,LastDate,price,mom,yoy]+list(df[i].iloc[-13:]), index=CommodityTable.columns)
    CommodityTable = CommodityTable.append(a_series, ignore_index=True)

for i in ColsOthers:
    cat="Others"
    name=i
    units=df[i].iloc[0]
    LastDate=df[i].tail(1).index[0]
    price=float(df[i].tail(1))
    mom=(df[i].iloc[-1] -df[i].iloc[-2])/df[i].iloc[-2]*100
    yoy=(df[i].iloc[-1] -df[i].iloc[-13])/df[i].iloc[-13]*100
    a_series = pd.Series([cat,name,units,LastDate,price,mom,yoy]+list(df[i].iloc[-13:]), index=CommodityTable.columns)
    CommodityTable = CommodityTable.append(a_series, ignore_index=True)
    
    

for i in ColsFert:
    cat="Fertilizers"
    name=i
    units=df[i].iloc[0]
    LastDate=df[i].tail(1).index[0]
    price=float(df[i].tail(1))
    mom=(df[i].iloc[-1] -df[i].iloc[-2])/df[i].iloc[-2]*100
    yoy=(df[i].iloc[-1] -df[i].iloc[-13])/df[i].iloc[-13]*100
    a_series = pd.Series([cat,name,units,LastDate,price,mom,yoy]+list(df[i].iloc[-13:]), index=CommodityTable.columns)
    CommodityTable = CommodityTable.append(a_series, ignore_index=True)
    

headers = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.182 Safari/537.36"}
URL="https://tradingeconomics.com/commodity/baltic"
response=requests.get(URL,headers=headers)
soup=BeautifulSoup(response.content,'html.parser')
tables=soup.find_all('table','table')
   
df1=pd.read_html(str(tables[0]))[0]
mom=str(df1[df1['Unnamed: 0']=="Baltic Dry"]['Month'][0])
yoy=str(df1[df1['Unnamed: 0']=="Baltic Dry"]['Year'][0])
lastdate=str(df1[df1['Unnamed: 0']=="Baltic Dry"]['Date'][0])
price=str(df1[df1['Unnamed: 0']=="Baltic Dry"]['Price'][0])

CommodityTable=CommodityTable.append({'Group':"Logistics", 'Title': 'Baltic Dry Index','Units': "-", 
                       'LastDate':lastdate,'Price':price,'MoM':mom, 'YoY':yoy},ignore_index=True)

,Group,Title,Units,LastDate,Price,MoM,YoY,0,1,2,3,4,5,6,7,8,9,10,11,12
0,Energy,"Crude oil, average",($/bbl),2022M05,110.096667,6.462739,65.808233,66.400000,71.803333,73.283333,68.866667,72.800000,82.063333,79.916667,72.866667,83.920000,93.543333,112.396667,103.413333,110.096667
1,Energy,"Natural gas, US",($/mmbtu),2022M05,8.1373,24.60264,181.723446,2.888400,3.231800,3.803600,4.048600,5.111100,5.478,5.017600,3.732700,4.332500,4.6577,4.883900,6.530600,8.137300
2,Oil,Palm oil,($/mt),2022M05,1716.92,2.031211,51.076149,1136.460000,1004.420000,1062.990000,1141.820000,1181.380000,1310.25,1340.650000,1270.290000,1344.790000,1522.36,1776.960000,1682.740000,1716.920000
3,Oil,Soybean oil,($/mt),2022M05,1962.88,0.789213,24.653419,1574.670000,1518.160000,1468.340000,1433.940000,1398.750000,1483.52,1442.960000,1411.210000,1469.560000,1595.74,1956.880000,1947.510000,1962.880000
4,Oil,Sunflower oil,($/mt),2022M05,2079.3,-8.63272,31.220335,1584.586721,1296.747967,1282.012195,1355.691057,1309.518970,1420.528455,1415.616531,1361.830962,1411.730000,1499.12,2361.130000,2275.760000,2079.300000
5,Cereals and Grains,Soybeans,($/mt),2022M05,724.09,0.457831,11.949598,646.800000,614.680000,600.440000,585.800000,557.550000,551.95,551.040000,554.140000,606.220000,661.63,720.600000,720.790000,724.090000
6,Cereals and Grains,Maize,($/mt),2022M05,344.840059,-0.955461,12.947275,305.310650,292.559355,278.434117,256.612434,235.617480,239.648763,248.719150,264.537213,276.623189,292.622344,335.529527,348.166655,344.840059
7,Cereals and Grains,"Rice, Thai 5%",($/mt),2022M05,464.0,7.656613,-5.882353,493.000000,466.000000,414.000000,403.000000,400.000000,401.0,400.000000,400.000000,427.000000,427.0,422.000000,431.000000,464.000000
8,Cereals and Grains,"Rice, Thai 25%",($/mt),2022M05,458.0,6.759907,-5.175983,483.000000,455.000000,407.000000,392.000000,391.000000,395.0,394.000000,393.000000,419.000000,422.0,419.000000,429.000000,458.000000
9,Cereals and Grains,"Rice, Thai A.1",($/mt),2022M05,442.86,8.262847,-4.298217,462.750000,438.590000,397.000000,381.000000,381.270000,382.71,378.550000,380.950000,403.150000,405.95,407.130000,409.060000,442.860000


In [146]:
#Create Outputs for the Dashboard to consume

import datetime as dt
datestamp=str(dt.datetime.today().month) + "-"+ str(dt.datetime.today().year)
CommodityTable.to_csv(os.path.join(os.getcwd(),"outputs","CommodityData_"+datestamp+".csv"))
NewData.to_csv(os.path.join(os.getcwd(),"outputs","DMCData_"+datestamp+".csv"))


In [145]:
NewData

,name,ISO,ADB_DEP,TE_Name,CurrencyCode,currency_Price,currency_YOYChange,currency_Date,CPI_Inflation_YoY,Inflation_Change,CPI_Inflation_Date,Food_Inflation_YoY,Food_Inflation_Change,Food_Inflation_Date
0,Afghanistan,AFG,CWRD,Afghanistan,AFN,88.66,14.02%,Jun/10,1.56,0.21,Jun 2021,-0.4,1.1,Jun 2021
1,Armenia,ARM,CWRD,Armenia,AMD,420.95,-18.67%,Jun/10,9.0,0.6,May 2022,14.5,0.2,May 2022
2,Azerbaijan,AZE,CWRD,Azerbaijan,AZN,1.693,0%,Jun/10,12.4,0.2,Apr 2022,17.4,0.4,Apr 2022
3,Georgia,GEO,CWRD,Georgia,GEL,2.895,-7.95%,Jun/10,13.3,0.5,May 2022,22.0,0.7,May 2022
4,Kazakhstan,KAZ,CWRD,Kazakhstan,KZT,436.08,2.18%,Jun/10,14.0,0.8,May 2022,19.0,1.1,May 2022
5,Kyrgyzstan,KGZ,CWRD,Kyrgyzstan,KGS,79.5013,-5.99%,Jun/10,14.5,1.3,Apr 2022,17.9,2.1,Apr 2022
6,Pakistan,PAK,CWRD,Pakistan,PKR,201.6,29.44%,Jun/10,13.8,0.4,May 2022,17.25,0.21,May 2022
7,Tajikistan,TJK,CWRD,Tajikistan,TJS,11.1,-1.86%,Jun/10,7.3,0.0,Apr 2022,8.1,1.0,Apr 2022
8,Turkmenistan,TKM,CWRD,Turkmenistan,TMT,3.49,0%,Jun/10,12.45,4.87,Dec 2021,,,
9,Uzbekistan,UZB,CWRD,Uzbekistan,UZS,11000.03,3.85%,Jun/10,10.95,0.57,May 2022,14.3,-2.9,Dec 2021


In [132]:
price

'2320.0'